# anno_dict, a possible issue there?

In [ ]:
from fastcore.meta import *
from fastcore.test import *
import inspect

## `anno_dict` docs

In [ ]:
inspect.getdoc(anno_dict)

"`__annotation__ dictionary with `empty` cast to `None`, returning empty if doesn't exist"

I have to confess I don't undersatnd the docs statement very well. So, I look into the source code of `anno_dict` and `empty2none`.

In [ ]:
print(inspect.getsource(anno_dict))

def anno_dict(f):
    "`__annotation__ dictionary with `empty` cast to `None`, returning empty if doesn't exist"
    return {k:empty2none(v) for k,v in getattr(f, '__annotations__', {}).items()}



In [ ]:
print(inspect.getsource(empty2none))

def empty2none(p):
    "Replace `Parameter.empty` with `None`"
    return None if p==inspect.Parameter.empty else p



## Dive in

If a parameter's default value is `Parameter.empty`, then `empty2none` is to replace `Parameter.empty` with `None` . So, I think it is reasonable to assume `p` is primarily used as a parameter's default value. The cell below supports this assumption.

In [ ]:
def foo(a, b:int=1): pass
sig = inspect.signature(foo)
for k,v in sig.parameters.items():
    print(f'{k} is a parameter {v}, whose default value is {v.default}, \
if apply empty2none to default value, then the default value is {empty2none(v.default)}')
    print(f'{k} is a parameter {v}, whose default value is {v.default}, \
if apply empty2none to parameter, then we get: {empty2none(v)}')

a is a parameter a, whose default value is <class 'inspect._empty'>, if apply empty2none to default value, then the default value is None
a is a parameter a, whose default value is <class 'inspect._empty'>, if apply empty2none to parameter, then we get: a
b is a parameter b: int = 1, whose default value is 1, if apply empty2none to default value, then the default value is 1
b is a parameter b: int = 1, whose default value is 1, if apply empty2none to parameter, then we get: b: int = 1


So, **what is odd** is that in `anno_dict`, `empty2none` is applied to `v` which is not parameter's default value, but mostly classes like `int`, `list` ect, as in `__annotations__`.

Then I experimented the section below and didn't find `anno_dict` doing anything new than `__annotations__`. 



## `anno_dict` seems not add anything new to `__annotations__`

In [ ]:
def foo(a, b:int=1): pass
test_eq(foo.__annotations__, {'b': int})
test_eq(anno_dict(foo), {'b': int})
def foo(a:bool, b:int=1): pass
test_eq(foo.__annotations__, {'a': bool, 'b': int})
test_eq(anno_dict(foo), {'a': bool, 'b': int})
def foo(a, d:list, b:int=1, c:bool=True): pass
test_eq(foo.__annotations__, {'d': list, 'b': int, 'c': bool})
test_eq(anno_dict(foo), {'d': list, 'b': int, 'c': bool})

In [ ]:
from fastcore.foundation import L

In [ ]:
def foo(a, b): pass
test_eq(foo.__annotations__, {})
test_eq(anno_dict(foo), {})

def _f(a:int, b:L)->str: ...
test_eq(_f.__annotations__, {'a': int, 'b': L, 'return': str})
test_eq(anno_dict(_f), {'a': int, 'b': L, 'return': str})

**Question!** so far above anno_dict has done nothing new or more, so what am I missing here?

## use fastdebug to double check

In [ ]:
from fastdebug.utils import *
from fastdebug.core import *
from fastcore.meta import *

In [ ]:
fdb = Fastdb(anno_dict)
fdb.eg = """
def foo(a, b): pass
test_eq(foo.__annotations__, {})
test_eq(anno_dict(foo), {})

from fastcore.foundation import L
def _f(a:int, b:L)->str: ...
test_eq(_f.__annotations__, {'a': int, 'b': L, 'return': str})
test_eq(anno_dict(_f), {'a': int, 'b': L, 'return': str})
"""

In [ ]:
fdb.snoop(['empty2none(v)'])

15:00:08.27 >>> Call to anno_dict in File "/tmp/anno_dict.py", line 3
15:00:08.27 ...... f = <function foo>
15:00:08.27    3 | def anno_dict(f):
15:00:08.27    5 |     return {k:empty2none(v) for k,v in getattr(f, '__annotations__', {}).items()}
    15:00:08.27 Dict comprehension:
    15:00:08.27    5 |     return {k:empty2none(v) for k,v in getattr(f, '__annotations__', {}).items()}
    15:00:08.27 .......... Iterating over <dict_itemiterator object>
    15:00:08.27 Result: {}
15:00:08.27    5 |     return {k:empty2none(v) for k,v in getattr(f, '__annotations__', {}).items()}
15:00:08.27 <<< Return value from anno_dict: {}
15:00:08.27 >>> Call to anno_dict in File "/tmp/anno_dict.py", line 3
15:00:08.27 ...... f = <function _f>
15:00:08.27    3 | def anno_dict(f):
15:00:08.27    5 |     return {k:empty2none(v) for k,v in getattr(f, '__annotations__', {}).items()}
    15:00:08.27 Dict comprehension:
    15:00:08.27    5 |     return {k:empty2none(v) for k,v in getattr(f, '__annotations

========================================================     Investigating anno_dict     =========================================================
==============================================================     on line None     ==============================================================
     with example 
def foo(a, b): pass
test_eq(foo.__annotations__, {})
test_eq(anno_dict(foo), {})

from fastcore.foundation import L
def _f(a:int, b:L)->str: ...
test_eq(_f.__annotations__, {'a': int, 'b': L, 'return': str})
test_eq(anno_dict(_f), {'a': int, 'b': L, 'return': str})
     



In [ ]:
fdb.docsrc(1, "empty2none works on paramter.default especially when the default is Parameter.empty; anno_dict works on the types \
of params, not the value of params; so it is odd to use empty2none in anno_dict;")

========================================================     Investigating anno_dict     =========================================================
===============================================================     on line 1     ================================================================
     with example 
def foo(a, b): pass
test_eq(foo.__annotations__, {})
test_eq(anno_dict(foo), {})

from fastcore.foundation import L
def _f(a:int, b:L)->str: ...
test_eq(_f.__annotations__, {'a': int, 'b': L, 'return': str})
test_eq(anno_dict(_f), {'a': int, 'b': L, 'return': str})
     

print selected srcline with expands below--------
def anno_dict(f):                                                                                                                                       (0)
    "`__annotation__ dictionary with `empty` cast to `None`, returning empty if doesn't exist"==========================================================(1)
empty2none works on paramter.default especially when

In [ ]:
fdb.print()

========================================================     Investigating anno_dict     =========================================================
===============================================================     on line 1     ================================================================
     with example 
def foo(a, b): pass
test_eq(foo.__annotations__, {})
test_eq(anno_dict(foo), {})

from fastcore.foundation import L
def _f(a:int, b:L)->str: ...
test_eq(_f.__annotations__, {'a': int, 'b': L, 'return': str})
test_eq(anno_dict(_f), {'a': int, 'b': L, 'return': str})
     

def anno_dict(f):=========================================================================(0)       
    "`__annotation__ dictionary with `empty` cast to `None`, returning empty if doesn't exist" # empty2none works on paramter.default especially when the default is Parameter.empty; anno_dict works on the types of params, not the value of params; so it is odd to use empty2none in anno_dict;;  (1)
    return {k:e

## Does fastcore want anno_dict to include params with no annos?

If so, I have written a lengthy `anno_dict_maybe` to do it. (can be shorter if needed)

In [ ]:
def anno_dict_maybe(f):
    "`__annotation__ dictionary with `empty` cast to `None`, returning empty if doesn't exist"
    new_anno = {}
    for k, v in inspect.signature(f).parameters.items():
        if k not in f.__annotations__:
            new_anno[k] = None
        else: 
            new_anno[k] = f.__annotations__[k]
    if 'return' in f.__annotations__:
        new_anno['return'] = f.__annotations__['return']
    if hasattr(f, '__annotations__'):
        return new_anno
    else:
        return {}

In [ ]:
def foo(a:int, b, c:bool=True)->str: pass

In [ ]:
test_eq(foo.__annotations__, {'a': int, 'c': bool, 'return': str})

In [ ]:
test_eq(anno_dict(foo), {'a': int, 'c': bool, 'return': str})

In [ ]:
test_eq(anno_dict_maybe(foo), {'a': int, 'b': None, 'c': bool, 'return': str})

In [ ]:
def foo(a, b, c): pass

In [ ]:
test_eq(foo.__annotations__, {})

In [ ]:
test_eq(anno_dict(foo), {})

In [ ]:
test_eq(anno_dict_maybe(foo), {})

## Jeremy's response

A supportive and confirmative [response](https://forums.fast.ai/t/help-reading-fastcore-docs/100168/3?u=daniel) from Jeremy on this issue